In [1]:
import duckdb
import pandas as pd

con = duckdb.connect("../data_processed/nyc_taxis.duckdb")


In [3]:
con.execute("SHOW TABLES").fetchall()
con.execute("SELECT COUNT(*) FROM trips_all").fetchall()


[(34499859,)]

In [10]:
df = con.execute("""
SELECT *
FROM trips_all
LIMIT 10000
""").fetchdf()

df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01,2016-01-01,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01,2016-01-01,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01,2016-01-01,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01,2016-01-01,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01,2016-01-01,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8


In [ ]:
con.execute("SELECT COUNT(*) FROM trips_all").fetchall()


[(34499859,)]

In [ ]:
con.execute("""                     
        DESCRIBE trips_all     
            """).fetchall()

[('VendorID', 'BIGINT', 'YES', None, None, None),
 ('tpep_pickup_datetime', 'TIMESTAMP', 'YES', None, None, None),
 ('tpep_dropoff_datetime', 'TIMESTAMP', 'YES', None, None, None),
 ('passenger_count', 'BIGINT', 'YES', None, None, None),
 ('trip_distance', 'DOUBLE', 'YES', None, None, None),
 ('pickup_longitude', 'DOUBLE', 'YES', None, None, None),
 ('pickup_latitude', 'DOUBLE', 'YES', None, None, None),
 ('RatecodeID', 'BIGINT', 'YES', None, None, None),
 ('store_and_fwd_flag', 'VARCHAR', 'YES', None, None, None),
 ('dropoff_longitude', 'DOUBLE', 'YES', None, None, None),
 ('dropoff_latitude', 'DOUBLE', 'YES', None, None, None),
 ('payment_type', 'BIGINT', 'YES', None, None, None),
 ('fare_amount', 'DOUBLE', 'YES', None, None, None),
 ('extra', 'DOUBLE', 'YES', None, None, None),
 ('mta_tax', 'DOUBLE', 'YES', None, None, None),
 ('tip_amount', 'DOUBLE', 'YES', None, None, None),
 ('tolls_amount', 'DOUBLE', 'YES', None, None, None),
 ('improvement_surcharge', 'DOUBLE', 'YES', None, Non

In [ ]:
# Durée des courses:
con.execute("""  
         SELECT 
    AVG(DATEDIFF('minute', tpep_pickup_datetime, tpep_dropoff_datetime)) AS duree_moyenne,
    MIN(DATEDIFF('minute', tpep_pickup_datetime, tpep_dropoff_datetime)) AS duree_min,
    MAX(DATEDIFF('minute', tpep_pickup_datetime, tpep_dropoff_datetime)) AS duree_max
FROM trips_all; 
            """).fetchall()

[(15.498414848594019, -537100, 192313)]

In [ ]:
# Demande par heure(pickup)
con.execute("""SELECT 
    EXTRACT(HOUR FROM tpep_pickup_datetime) AS heure,
    COUNT(*) AS nb_courses
FROM trips_all
GROUP BY heure
ORDER BY heure;
  
            """).fetchall()

[(0, 1221488),
 (1, 891199),
 (2, 650316),
 (3, 490617),
 (4, 370616),
 (5, 349856),
 (6, 765913),
 (7, 1300849),
 (8, 1593081),
 (9, 1593849),
 (10, 1539389),
 (11, 1594604),
 (12, 1690509),
 (13, 1690232),
 (14, 1769355),
 (15, 1741841),
 (16, 1579778),
 (17, 1855814),
 (18, 2186699),
 (19, 2171514),
 (20, 2014201),
 (21, 1972162),
 (22, 1878174),
 (23, 1587803)]

In [ ]:
# Demande par jour de semaine
con.execute("""SELECT 
    STRFTIME(tpep_pickup_datetime, '%w') AS jour,
    COUNT(*) AS nb_courses
FROM trips_all
GROUP BY jour
ORDER BY jour; 
            """).fetchall()

[('0', 4556265),
 ('1', 4433368),
 ('2', 4788480),
 ('3', 4994465),
 ('4', 5230066),
 ('5', 5370914),
 ('6', 5126301)]

In [ ]:
# Moyenne distance trajet+prix
con.execute("""SELECT 
    AVG(trip_distance) AS distance_moyenne,
    AVG(total_amount) AS montant_moyen
FROM trips_all;
            """).fetchall()

[(5.309407807434687, 15.771750543611502)]

In [ ]:
#Distance --> Montant
con.execute("""SELECT 
    ROUND(trip_distance) AS distance_arrondie,
    AVG(total_amount) AS montant_moyen
FROM trips_all
GROUP BY distance_arrondie
ORDER BY distance_arrondie
LIMIT 20;
      
            """).fetchall()

[(-3390584.0, 4.3),
 (0.0, 8.875042789713774),
 (1.0, 8.68400708009996),
 (2.0, 12.584574738946985),
 (3.0, 16.31456262481267),
 (4.0, 19.7129011027755),
 (5.0, 22.85044275208809),
 (6.0, 25.84996332036184),
 (7.0, 28.882023831885874),
 (8.0, 32.85845282480317),
 (9.0, 37.79049926322786),
 (10.0, 41.26717731740262),
 (11.0, 44.2617352328683),
 (12.0, 46.50714469056725),
 (13.0, 49.82595081301641),
 (14.0, 53.30148564308816),
 (15.0, 56.690445302378485),
 (16.0, 61.619723102006574),
 (17.0, 65.58141024394564),
 (18.0, 66.11388698828881)]

In [ ]:
#Pourboires par types de paiement
con.execute("""
            SELECT 
    payment_type,
    AVG(tip_amount) AS tip_moyen,
    COUNT(*) AS nb
FROM trips_all
GROUP BY payment_type
ORDER BY nb DESC;      
            
            """).fetchall()

[(1, 2.660161064143558, 22989226),
 (2, 0.0001504819552681602, 11341094),
 (3, 0.004200356955045975, 125506),
 (4, -0.005736282703488374, 44032),
 (5, 0.0, 1)]

In [ ]:
# Montant négatifs
con.execute("""                     
  SELECT COUNT(*)
FROM trips_all
WHERE tpep_dropoff_datetime < tpep_pickup_datetime;
            """).fetchall()

[(176,)]

In [ ]:
con.execute("""                      
        SELECT COUNT(*)
FROM trips_all
WHERE trip_distance < 0;         
            """).fetchall()

[(1,)]

In [ ]:
con.execute("""                                
       SELECT COUNT(*)
FROM trips_all
WHERE total_amount < 0;
            """).fetchall()

[(13090,)]

In [ ]:
# Géographie(pickup)
con.execute("""                     
          SELECT 
    ROUND(pickup_latitude, 2) AS lat,
    ROUND(pickup_longitude, 2) AS lon,
    COUNT(*) AS nb
FROM trips_all
GROUP BY lat, lon
ORDER BY nb DESC
LIMIT 20;
            """).fetchall()

[(40.76, -73.97, 2119190),
 (40.75, -73.99, 2111204),
 (40.76, -73.98, 1857031),
 (40.75, -73.98, 1716875),
 (40.74, -73.99, 1560441),
 (40.76, -73.99, 1555088),
 (40.73, -73.99, 1227937),
 (40.77, -73.96, 1200566),
 (40.77, -73.98, 1099698),
 (40.73, -74.0, 1038917),
 (40.74, -74.0, 1024452),
 (40.74, -73.98, 986659),
 (40.78, -73.96, 893269),
 (40.78, -73.95, 886134),
 (40.78, -73.98, 843673),
 (40.72, -73.99, 790466),
 (40.75, -73.97, 759209),
 (40.77, -73.95, 648756),
 (40.72, -74.0, 647825),
 (40.76, -73.96, 596616)]

In [ ]:
con.execute("""                     
            
            
            
            
            
            
            
            """).fetchall()

In [ ]:
con.execute("""                     
            
            
            
            
            
            
            
            """).fetchall()

In [ ]:
con.execute("""                     
            
            
            
            
            
            
            
            """).fetchall()

In [ ]:
con.execute("""                     
            
            
            
            
            
            
            
            """).fetchall()